In [1]:
import os
import subprocess
import pandas as pd
import json

import viame2coco.viame2coco as viame2coco
from pycocotools.coco import COCO
import cv2
# import matplotlib.pyplot as plt
import math


In [11]:
imagery_raw_bucket = "amlr-gliders-imagery-raw-dev"
imagery_raw_mt = "/home/sam_woodman_noaa_gov/amlr-gliders-imagery-raw-dev"

lib_bucket = "esd-image-library-dev"
lib_mt = "/home/sam_woodman_noaa_gov/esd-image-library-dev"

csv_base_path = "/home/sam_woodman_noaa_gov/jw-annotations/amlr08-20220513"

# Mount bucket(s)
os.makedirs(imagery_raw_mt, exist_ok=True)
cmd = ["gcsfuse", "--implicit-dirs", "-o", "ro", imagery_raw_bucket, imagery_raw_mt]
subprocess.run(cmd)

os.makedirs(lib_mt, exist_ok=True)
cmd = ["gcsfuse", "--implicit-dirs", lib_bucket, lib_mt]
subprocess.run(cmd)

{"timestamp":{"seconds":1743104030,"nanos":117764900},"severity":"INFO","message":"Start gcsfuse/2.9.0 (Go version go1.23.5) for app \"\" using mount point: /home/sam_woodman_noaa_gov/amlr-gliders-imagery-raw-dev\n"}
{"timestamp":{"seconds":1743104030,"nanos":117796076},"severity":"INFO","message":"GCSFuse config","config":{"AppName":"","CacheDir":"","Debug":{"ExitOnInvariantViolation":false,"Fuse":false,"Gcs":false,"LogMutex":false},"EnableAtomicRenameObject":false,"EnableHns":true,"FileCache":{"CacheFileForRangeRead":false,"DownloadChunkSizeMb":50,"EnableCrc":false,"EnableODirect":false,"EnableParallelDownloads":false,"MaxParallelDownloads":16,"MaxSizeMb":-1,"ParallelDownloadsPerFile":16,"WriteBufferSize":4194304},"FileSystem":{"DirMode":"755","DisableParallelDirops":false,"FileMode":"644","FuseOptions":["ro"],"Gid":-1,"HandleSigterm":true,"IgnoreInterrupts":true,"KernelListCacheTtlSecs":0,"PreconditionErrors":true,"RenameDirLimit":0,"TempDir":"","Uid":-1},"Foreground":false,"GcsAuth

CompletedProcess(args=['gcsfuse', '--implicit-dirs', 'esd-image-library-dev', '/home/sam_woodman_noaa_gov/esd-image-library-dev'], returncode=0)

## Convert VIAME CSV file to COCO format

In [4]:
# Read in CSV file(s)
os.listdir(csv_base_path)

csv_path2 = os.path.join(csv_base_path, 'amlr08-20220513-dir0002-annotations-manual.csv')

d2 = pd.read_csv(csv_path2)
display(d2)

display(d2.iloc[1:, 9].value_counts())

# pref = "/home/sam_woodman_noaa_gov/Dir0002"
# d2.loc[1:, '2: Video or Image Identifier'] = (
#     [f"{pref}/{i}" for i in d2['2: Video or Image Identifier'].values[1:]])
# d2.to_csv("tmp2.csv", index=False)

,# 1: Detection or Track-id,2: Video or Image Identifier,3: Unique Frame Identifier,4-7: Img-bbox(TL_x,TL_y,BR_x,BR_y),8: Detection or Length Confidence,9: Target Length (0 or -1 if invalid),10-11+: Repeated Species,Confidence Pairs or Attributes
0,# metadata,"exported_by: ""dive:python""","exported_time: ""Mon Dec 9 23:56:55 2024""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,SG01 20220513-205705-001.jpg,0,1719.0,502.0,1787.0,595.0,1.0,-1.0,copepod,1.0
2,1,SG01 20220513-205736-011.jpg,10,1916.0,1893.0,2016.0,1988.0,1.0,-1.0,copepod,1.0
3,2,SG01 20220513-205739-012.jpg,11,2050.0,1107.0,2166.0,1257.0,1.0,-1.0,copepod,1.0
4,3,SG01 20220513-205742-013.jpg,12,3199.0,1578.0,3335.0,1719.0,1.0,-1.0,copepod,1.0
...,...,...,...,...,...,...,...,...,...,...,...
456,455,SG01 20220513-214016-018.jpg,757,1360.0,2141.0,1458.0,2280.0,1.0,-1.0,copepod,1.0
457,456,SG01 20220513-214019-019.jpg,758,1297.0,2318.0,1487.0,2504.0,1.0,-1.0,hydrozoan,1.0
458,457,SG01 20220513-214019-019.jpg,758,2419.0,2061.0,2538.0,2165.0,1.0,-1.0,copepod,1.0
459,458,SG01 20220513-214022-020.jpg,759,1912.0,1827.0,2009.0,1962.0,1.0,-1.0,copepod,1.0


10-11+: Repeated Species
copepod         164
radiolarian     163
larvacean        35
unid             34
chaetognath      19
hydrozoan        15
siphonophore     11
ctenophore        6
fish              4
egg               4
echinoderm        2
pteropod          2
euphausiid        1
Name: count, dtype: int64

In [5]:
# viame2coco.viame2coco(csv_path2, "temporary description")
dir_path = os.path.join(
        f"/home/sam_woodman_noaa_gov/{imagery_raw_bucket}", 
        "SANDIEGO/2022/amlr08-20220513/images/Dir0002")

config = {
    'filename': 1,
    'label': 9, 
    'bbox_tlbr': {
        'tlx': 3,
        'tly': 4,
        'brx': 5,
        'bry': 6
    }
}

c2 = viame2coco.viame2coco(
    csv_path2, "temporary description", 
    filename_base=dir_path, viame_csv_config=config
)
c2.to_json("../tmp2-coco.json")
display(c2)

### Explore COCO output

In [6]:
with open("../tmp2-coco.json") as fin:
    j2 = json.load(fin)

print(j2.keys())
[print(j2[i]) for i in j2.keys()]

# j2_images = j2["images"]
# print(j2_images)

# j2_anno = j2["annotations"]
# print(j2_anno)

# # print(j2["info"])
# # print(j2["licenses"])
# # print(j2["categories"])

j2_cat = j2["categories"]
j2_cat

dict_keys(['info', 'images', 'annotations', 'licenses', 'categories'])
{'year': 2025, 'version': '0.1', 'description': 'temporary description', 'date_created': '2025-03-27T19:07:48.705577+00:00'}
[{'id': 1, 'file_name': 'SG01 20220513-205705-001.jpg', 'width': 4056, 'height': 3040}, {'id': 2, 'file_name': 'SG01 20220513-205736-011.jpg', 'width': 4056, 'height': 3040}, {'id': 3, 'file_name': 'SG01 20220513-205739-012.jpg', 'width': 4056, 'height': 3040}, {'id': 4, 'file_name': 'SG01 20220513-205742-013.jpg', 'width': 4056, 'height': 3040}, {'id': 5, 'file_name': 'SG01 20220513-205816-002.jpg', 'width': 4056, 'height': 3040}, {'id': 6, 'file_name': 'SG01 20220513-205847-012.jpg', 'width': 4056, 'height': 3040}, {'id': 7, 'file_name': 'SG01 20220513-205902-017.jpg', 'width': 4056, 'height': 3040}, {'id': 8, 'file_name': 'SG01 20220513-205940-007.jpg', 'width': 4056, 'height': 3040}, {'id': 9, 'file_name': 'SG01 20220513-205952-011.jpg', 'width': 4056, 'height': 3040}, {'id': 10, 'file_nam

[{'name': 'copepod', 'id': 1},
 {'name': 'larvacean', 'id': 2},
 {'name': 'ctenophore', 'id': 3},
 {'name': 'chaetognath', 'id': 4},
 {'name': 'fish', 'id': 5},
 {'name': 'hydrozoan', 'id': 6},
 {'name': 'unid', 'id': 7},
 {'name': 'radiolarian', 'id': 8},
 {'name': 'echinoderm', 'id': 9},
 {'name': 'siphonophore', 'id': 10},
 {'name': 'egg', 'id': 11},
 {'name': 'euphausiid', 'id': 12},
 {'name': 'pteropod', 'id': 13}]

## Extract regions from images
 
Using COCO output, based on this code: https://forum.image.sc/t/crop-image-and-annotations-to-bbox-coco-format/74520/4

COCO data format: https://cocodataset.org/#format-data

COCO API functions: top of https://github.com/ppwwyyxx/cocoapi/blob/master/PythonAPI/pycocotools/coco.py

In [ ]:
### Sam exp - loop through categories and write regions to category folders

deployment = 'amlr08-20220513'
roi_base = os.path.join(lib_mt, "esd-shadowgraph-library")
# roi_base = os.path.join("/home/sam_woodman_noaa_gov", "Dir0002-out")

# Load COCO file for images+annotations+categories
annFile = "../tmp2-coco.json"
coco=COCO(annFile) #load via cocoapi
cats = coco.loadCats(coco.getCatIds()) #get all categorys in coco file

cats_names = [i["name"] for i in cats]
print(f"Categories: {cats_names}")

for cat in cats:
    # Get category name, and make directory
    cat_name = cat["name"]
    cat_base = os.path.join(roi_base, cat_name)
    os.makedirs(cat_base, exist_ok=True)
    
    catIds = coco.getCatIds(catNms=[cat_name]) #use to pull cat ID from cat
    imgIds = coco.getImgIds(catIds=catIds ) #get all images with the specified category above in

    print(f"For category {cat_name}, there are {len(imgIds)} images. " +
            f"These are being written to {cat_base}")
    for img_idx in imgIds:
        # Get and read in the current image
        img = coco.loadImgs(img_idx)[0]
        I = cv2.imread(os.path.join(dir_path, img['file_name']))
        # I

        # Get the current annotation(s)
        annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
        anns = coco.loadAnns(annIds)

        for i in range(len(anns)):
            ann = anns[i]
            crop_bbox = ann["bbox"]

            #make cropped image
            cropim = I[math.floor(crop_bbox[1]):math.ceil(crop_bbox[1] + crop_bbox[3]), 
                        math.floor(crop_bbox[0]):math.ceil(crop_bbox[0]+crop_bbox[2])]
            
            #save cropped image
            file_name_curr = os.path.splitext(img["file_name"])
            file_name = (
                f"{deployment}_{file_name_curr[0]}_crop-{(i+1):02}" + 
                file_name_curr[1])
            path_out = os.path.join(cat_base, file_name)
            cv2.imwrite(path_out, cropim)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Categories: ['copepod', 'larvacean', 'ctenophore', 'chaetognath', 'fish', 'hydrozoan', 'unid', 'radiolarian', 'echinoderm', 'siphonophore', 'egg', 'euphausiid', 'pteropod']
For category copepod, there are 136 images. These are being written to /home/sam_woodman_noaa_gov/esd-image-library-dev/esd-shadowgraph-library/copepod
{'image_id': 39, 'id': 42, 'category_id': 1, 'bbox': [1100, 692, 109, 79], 'area': 8611, 'iscrowd': 0}
SG01 20220513-210726-003.jpg
{'image_id': 70, 'id': 103, 'category_id': 1, 'bbox': [3040, 822, 114, 130], 'area': 14820, 'iscrowd': 0}
SG01 20220513-211001-009.jpg
{'image_id': 70, 'id': 104, 'category_id': 1, 'bbox': [937, 1038, 91, 91], 'area': 8281, 'iscrowd': 0}
SG01 20220513-211001-009.jpg
{'image_id': 70, 'id': 107, 'category_id': 1, 'bbox': [1522, 2475, 123, 129], 'area': 15867, 'iscrowd': 0}
SG01 20220513-211001-009.jpg
{'image_id': 72, 'id': 114, 'category_id': 1, 'bbox': [25

In [ ]:
# ### Sam exp - individual files
# q = 2

# # Get and read in the current image
# img = coco.loadImgs(imgIds[q])[0]
# I = cv2.imread(img['file_name'])
# I

# # Get the current annotation(s)
# annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
# anns = coco.loadAnns(annIds)

# crop_bbox = anns[0]["bbox"]
# #make cropped image
# cropim = I[math.floor(crop_bbox[1]):math.ceil(crop_bbox[1] + crop_bbox[3]), math.floor(crop_bbox[0]):math.ceil(crop_bbox[0]+crop_bbox[2])]
# #save cropped image
# cv2.imwrite(os.path.join("/home/sam_woodman_noaa_gov", os.path.basename(img["file_name"])), 
#             cropim)

In [ ]:
for q in range(0, len(imgIds)):#q is an image with interesting category in it
    img = coco.loadImgs(imgIds[q])[0]#load image name via cocoapi
    I = cv2.imread(os.path.join(working_dir, img['file_name'])) #load the image with cv2
    annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None) #Get image worm annotations
    anns = coco.loadAnns(annIds)
    embryo_annIds = coco.getAnnIds(imgIds=img['id'], catIds=[1], iscrowd=None) #Get image embryo annotations
    embryo_anns = coco.loadAnns(embryo_annIds)
    for worm_no in range(0, len(anns)):#worm_no is zero indexed #worms in image
        #dump dict looks like a java object, but it really a python dictionary we're going to be saving at the end
        #dump_dict = {"image": [], "categories": [], "annotations": [], }
        #Get the bbox from the worm to use as a crop box
        crop_bbox = anns[worm_no]["bbox"]
        #Get a list of embryos that are inside this crop box 
        embryos_in_worm = [embryo_anns[a] for a in range(0, len(embryo_anns)) if bbox_relation(crop_bbox, embryo_anns[a]['bbox']) == True]
        #make cropped image
        cropim = I[math.floor(crop_bbox[1]):math.ceil(crop_bbox[1] + crop_bbox[3]), math.floor(crop_bbox[0]):math.ceil(crop_bbox[0]+crop_bbox[2])]
        #save cropped image
        file_name_variable = img["file_name"] + "_worm_no_" + str(worm_no)
        cv2.imwrite(working_dir + file_name_variable, cropim)
        for b in range(0, len(embryos_in_worm)): #b is an embryo within the cropped image
            #Transpose embryo seg FIRST ALWAYS!! (function above)
            embryos_in_worm[b]['segmentation'] = seg_transpose(crop_bbox, embryos_in_worm[b]['segmentation'])
            #Transpose embryo bbox (function above - could also calculate from seg)
            embryos_in_worm[b]['bbox'] = bbox_transpose(crop_bbox, embryos_in_worm[b]['bbox'])
            #add new image id(assigning to image after!)
            embryos_in_worm[b]['image_id'] = new_im_id
            #add embryo annotation to dump_dict_annotations
            dump_dict["annotations"].append(embryos_in_worm[b])
        #Same but worm - only one per image probably, so no need for loop.
        worm_transpose_store = anns[worm_no]
        worm_transpose_store["image_id"] = new_im_id
        worm_transpose_store["segmentation"] = seg_transpose(anns[worm_no]["bbox"], worm_transpose_store["segmentation"])
        worm_transpose_store["bbox"] = bbox_transpose(anns[worm_no]["bbox"], anns[worm_no]["bbox"])
        dump_dict["annotations"].append(worm_transpose_store)
        #updating image dictionary for new json
        img_transpose_store = img
        img_transpose_store["height"] = crop_bbox[3]
        img_transpose_store["width"] = crop_bbox[2]
        img_transpose_store["id"] = new_im_id #id = count of worms from 0 - n-1
        img_transpose_store["file_name"] = file_name_variable
        #add image dictionary to dump_dict
        dump_dict["image"].append(img_transpose_store)
        #Update image number
        new_im_id += 1

#Straighten out any annotation duplications due to bbox overlap - duplicate annotations are a problem - deal with this via worm seg!!
for w in range(0, len(dump_dict["annotations"])):
    dump_dict['annotations'][w]["id"] = w
    
#Dump a single coco format json with all cropped images and annotations inside.
with open(working_dir + "worm_by_worm_embryo_test_transposed.json", "w") as outfile:
    json.dump(dump_dict, outfile, indent=4)